In [1]:
# !pip install -r requirements.txt

In [2]:
import asyncio
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from pathlib import Path
from flask import Flask, request
from contextlib import asynccontextmanager

# Define the path to your MCP server
server_params = StdioServerParameters(
    command="docker",
    args=[
        "run",
        "-i",
        "--rm",
        "-e",
        "GITHUB_PERSONAL_ACCESS_TOKEN",
        "ghcr.io/github/github-mcp-server"
      ],
    env={
        "GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_PERSONAL_ACCESS_TOKEN"),
    }
)

@asynccontextmanager
async def tool_session_generator():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            yield tools


async def ai(query: str):
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)
    async with tool_session_generator() as tools:
        agent_executor = create_react_agent(llm, tools)
    
        # System prompt allows for broad range of instructions. But what does user input look like?
        events = agent_executor.astream(
            {"messages": [("system", Path("web_server_context.txt").read_text()),
                          ("user", query)]},
            stream_mode="values",
        )
    
        async for event in events:
            event["messages"][-1].pretty_print()
            data = event["messages"][-1]
    
        return data.content


async def ai_act(query: str):
    act_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
    async with tool_session_generator() as tools:
        act_agent = create_react_agent(act_llm, tools)
    
        events = act_agent.astream(
            {"messages": [("system",
                           "You are a web server. Do not use markdown for formatting. Do exactly what the user asks. Do not use markdown in your response. Produce minimal output. Given my repo: https://github.com/breba-apps/TempRepo"),
                          ("user", query)]},
            stream_mode="values",
        )
        data = ""
        async for event in events:
            event["messages"][-1].pretty_print()
            data = event["messages"][-1]
    
        return data.content
        

app = Flask(__name__)
    

@app.route('/', methods=['GET'])
def get_data():
    query = Path("my_app-step-4-mcp.txt").read_text()
    response = asyncio.run(ai(query))
    return response


@app.route('/act', methods=['POST'])
def act():
    payload = request.data.decode('utf-8')
    print(request)
    response = asyncio.run(ai_act(payload))
    return response


app.run(debug=False, port=5001, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


<Request 'http://127.0.0.1:5001/act' [POST]>
================================ Human Message =================================

Update assignee for the issue.
Issue ID: 2, Assignee: yasonk
================================== Ai Message ==================================
Tool Calls:
  update_issue (call_peJB6v6SPB7kZeahowk35yM7)
 Call ID: call_peJB6v6SPB7kZeahowk35yM7
  Args:
    assignees: ['yasonk']
    issue_number: 2
    owner: breba-apps
    repo: TempRepo
================================= Tool Message =================================
Name: update_issue

Error: McpError('failed to update issue: PATCH https://api.github.com/repos/breba-apps/TempRepo/issues/2: 403 Resource not accessible by personal access token []')
 Please fix your mistakes.


127.0.0.1 - - [16/May/2025 22:30:18] "POST /act HTTP/1.1" 200 -


================================== Ai Message ==================================

The issue could be due to insufficient permissions with the current access token. Please ensure that the token has the necessary scope to update issues.


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~